Parameters and functions

In [21]:
import numpy as np

# Parameters
n = 4
k = 1
q = 13

def modular_mult(A, s):
    """Multiplies a matrix A by a scalar s modulo q."""
    return np.mod(A * s, q)

def modular_add(A, B):
    """Adds two matrices A and B modulo q."""
    return np.mod(A + B, q)

<img src="../media/keygen.png" alt="keygen" width="400"/>

In [22]:
## KeyGen (Alice)
# A: Public key - Part 1
np.random.seed(0)
A = np.random.randint(q,size=(n, n)) # {0, 1, ..., q-1}
print("A:\n", A)

# s: Secret key
s = np.random.randint(3, size=(n, 1)) - 1 # {-1, 0, 1}
print("s:\n", s)

# e: Noise vector
e = np.random.randint(3, size=(n, 1)) - 1 # {-1, 0, 1}
print("e:\n", e)

# t: Public key - Part 2
t = modular_mult(A,s)

print("t:\n",t)

A:
 [[12  5  0  3]
 [11  3  7  9]
 [ 3  5  2  4]
 [ 7  6  8  8]]
s:
 [[-1]
 [ 1]
 [ 0]
 [ 1]]
e:
 [[ 1]
 [-1]
 [ 0]
 [ 0]]
t:
 [[ 1  8  0 10]
 [11  3  7  9]
 [ 0  0  0  0]
 [ 7  6  8  8]]


<img src="../media/enc.png" alt="keygen" width="400"/>

In [23]:
# Encryption (Bob)
r  = np.random.randint(3, size=(n)) - 1 # {-1, 0, 1}
e1 = np.random.randint(3, size=(n)) - 1 # {-1, 0, 1}
e2 = np.random.randint(3, size=(1)) - 1 # {-1, 0, 1}
m  = np.random.randint(2, size=(1))     # {0, 1}
print("m:\n", m)

u = modular_mult(r, A)
u = modular_add(u, e1)
print("u:\n", u)

v = modular_mult(r, t)
v = modular_add(v, e2)
v = modular_add(v, m*((q+1)>>1))
print("v:\n", v)

m:
 [0]
u:
 [[12 12  0  1]
 [12 12  6  1]
 [12 12 11  1]
 [12 12  5  1]]
v:
 [[12 12 12 12]
 [12 12  5 12]
 [12 12 12 12]
 [12 12  4 12]]


<img src="../media/dec.png" alt="keygen" width="400"/>

In [ ]:
# Decryption (Alice)
f1 = modular_mult(u, s)
f2 = modular_add(v, f1)
print("f2:\n", f2)

if(f2[0] > (q/4) and (f2[0] < (3*q/4))):
    f2[0] = 1
else:
    f2[0] = 0
print(f2[0]==m[0])


f2:
 [[ 0  0 12 11]
 [11 11 11  0]
 [12 12 12 12]
 [11 11  9  0]]
Type of f2[0]: <class 'numpy.ndarray'>
Shape of f2[0]: (4,)
First few values: [ 0  0 12 11]


Rest of the notes:  
<img src="../media/rest.png" alt="keygen" width="600"/>

In [ ]:
from hashlib import sha3_512, sha3_256

def crypto_kem_keypair(init_seed):
    buf = sha3_512(init_seed)
    publicseed = get_nbit_from_m(buf, 32*8, 256)
    noiseseed = get_nbit_from_m(buf, 32*8, 0)

    A = gen_matrix(publicseed, 0) # 2 sets with 2 vectors of 256 coefficients
    nonce = 0
    skpv = {} # 2 vectors of 256 coefficients
    e = {} # 2 vectors of 256 coefficients
    pkpv = {}
    for i in range(0, KYBER_K, 1):
        skpv[i] = poly_getnoise(noiseseed, nonce)
        nonce = nonce + 1

    for i in range(0, KYBER_K, 1):
        e[i] = poly_getnoise(noiseseed, nonce)
        nonce = nonce + 1

    for i in range(0, KYBER_K, 1):
        fwd_NIT(skpv[i])

    for i in range(0, KYBER_K, 1):
        fwd_NIT(e[i])
    
    for i in range(0, KYBER_K, 1):
        pkpv[i] = polyvec_pointwise_acc(A[i][:][:], skpv)

    for i in range(0, KYBER_K, 1):
        pkpv[i] = poly_add(pkpv[i], e[i])
        pkpv[i] = poly_reduce(pkpv[i])

    skpv_packed = {}
    skpv_packed = pack_sk(skpv)

    pkpv_packed = {}
    pkpv_packed = pack_pk(pkpv, publicseed)

    skpv_packed = skpv_packed + pkpv_packed + hex(sha3_256(int(pkpv_packed, 16)))[2:]

    return (skpv_packed, pkpv_packed)

NameError: name 'modular_mult' is not defined

In [ ]:
print(hex(init_seed))
(sk, pk) = crypto_kem_keypair(init_seed)
print("Secret Key: ", sk)
print("Public Key: ", pk)

In [ ]:
print(hex(enc_seed))

buf1 = sha3_256(get_nbit_from_m(enc_seed, 32*8, 32*8))
buf0 = sha3_256(int(pk, 16))
buf = buf1 * 2 ** 256 + buf0
kr_enc = sha3_512(buf)
ct = crypto_kem_enc(pk, kr_enc, buf)

kr_enc = sha3_512_32byte=hex(kr_enc)[2:66] # show the first 32 bytes
ss_Bob = shake_256(int(kr_enc_32byte+hex(sha3_256(int(ct, 16)))[2:],16),32)

print(hex(buf))
print(hex(kr_enc))
print("Cipher text: ", ct)
hex(ss_Bob)